In [1]:
!cp -r /kaggle/input/stage2-14/Loss-Gated-Learning /kaggle/working/


In [2]:
%cd /kaggle/input/stage2-14/Loss-Gated-Learning
!pip install -r ./utils/requirements.txt  


/kaggle/input/stage2-14/Loss-Gated-Learning
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.7 MB/s eta 0:00:00


In [3]:
%cd Stage2

/kaggle/input/stage2-14/Loss-Gated-Learning/Stage2


In [4]:
%%writefile /kaggle/working/Loss-Gated-Learning/Stage2/dataLoader.py

import glob, numpy, os, random, soundfile, torch, wave
from scipy import signal
from tools import *

def get_Loader(args, dic_label = None, cluster_only = False):
    # Get the loader for the cluster, batch_size is set as 1 to handlle the variable length input. Details see 1.2 part from here: https://github.com/TaoRuijie/TalkNet-ASD/blob/main/FAQ.md 
    clusterLoader = cluster_loader(**vars(args))
    clusterLoader = torch.utils.data.DataLoader(clusterLoader, batch_size = 1, shuffle = True, num_workers = args.n_cpu, drop_last = False)

    if cluster_only == True: # Only do clustering
        return clusterLoader
    # Get the loader for training
    trainLoader = train_loader(dic_label = dic_label, **vars(args))
    trainLoader = torch.utils.data.DataLoader(trainLoader, batch_size = args.batch_size, shuffle = True, num_workers = args.n_cpu, drop_last = True)

    return trainLoader, clusterLoader

class train_loader(object):
    def __init__(self, train_list, train_path, musan_path, rir_path, max_frames, dic_label, **kwargs):
        self.train_path = train_path
        self.max_frames = max_frames * 160 + 240 # Length of segment for training
        self.dic_label = dic_label # Pseudo labels dict
        self.noisetypes = ['noise','speech','music']
        self.noisesnr = {'noise':[0,15],'speech':[13,20],'music':[5,15]}
        self.numnoise = {'noise':[1,1], 'speech':[3,8], 'music':[1,1]}
        self.noiselist = {}
        augment_files   = glob.glob(os.path.join(musan_path,'*/*/*/*.wav'))
        for file in augment_files:
            if file.split('/')[-3] not in self.noiselist:
                self.noiselist[file.split('/')[-3]] = []
            self.noiselist[file.split('/')[-3]].append(file)
        self.rir_files  = glob.glob(os.path.join(rir_path,'*/*/*.wav'))
        self.data_list = []
        lines = open(train_list).read().splitlines()
        for index, line in enumerate(lines):
            file_name     = line.split()[1]
            self.data_list.append(file_name)

    def __getitem__(self, index):
        file = self.data_list[index] # Get the filename
        label = self.dic_label[file] # Load the pseudo label
        segments = self.load_wav(file = file) # Load the augmented segment
        segments = torch.FloatTensor(numpy.array(segments))
        return segments, label

    def load_wav(self, file):
        utterance, _ = soundfile.read(os.path.join(self.train_path, file)) # Read the wav file
        if utterance.shape[0] <= self.max_frames: # Padding if less than required length
            shortage = self.max_frames - utterance.shape[0]
            utterance = numpy.pad(utterance, (0, shortage), 'wrap')
        startframe = random.choice(range(0, utterance.shape[0] - (self.max_frames))) # Choose the startframe randomly
        segment = numpy.expand_dims(numpy.array(utterance[int(startframe):int(startframe)+self.max_frames]), axis = 0)

        if random.random() <= 0.5:
            segment = self.add_rev(segment, length = self.max_frames) # Rever
        if random.random() <= 0.5:
            segment = self.add_noise(segment, random.choice(['music', 'speech', 'noise']), length = self.max_frames) # Noise

        return segment[0]

    def __len__(self):
        return len(self.data_list)

    def add_rev(self, audio, length):
        rir_file    = random.choice(self.rir_files)
        rir, sr     = soundfile.read(rir_file)
        rir         = numpy.expand_dims(rir.astype(numpy.float),0)
        rir         = rir / numpy.sqrt(numpy.sum(rir**2))
        return signal.convolve(audio, rir, mode='full')[:,:length]

    def add_noise(self, audio, noisecat, length):
        clean_db    = 10 * numpy.log10(numpy.mean(audio ** 2)+1e-4)
        numnoise    = self.numnoise[noisecat]
        # print("numnoise: ", numnoise)
        # print("self.noiselist[noisecat]: ", self.noiselist[noisecat])
        if len(self.noiselist[noisecat]) == 1:
            noiselist   = random.sample(self.noiselist[noisecat], 1)
        else:
            noiselist   = random.sample(self.noiselist[noisecat], random.randint(numnoise[0],numnoise[1]))
        noises = []
        for noise in noiselist:
            noiselength = wave.open(noise, 'rb').getnframes() # Read the length of the noise file			
            if noiselength <= length:
                noiseaudio, _ = soundfile.read(noise)
                noiseaudio = numpy.pad(noiseaudio, (0, length - noiselength), 'wrap')
            else:
                start_frame = numpy.int64(random.random()*(noiselength-length)) # If length is enough
                noiseaudio, _ = soundfile.read(noise, start = start_frame, stop = start_frame + length) # Only read some part to improve speed
            noiseaudio = numpy.stack([noiseaudio],axis=0)
            noise_db = 10 * numpy.log10(numpy.mean(noiseaudio ** 2)+1e-4) 
            noisesnr   = random.uniform(self.noisesnr[noisecat][0],self.noisesnr[noisecat][1])
            noises.append(numpy.sqrt(10 ** ((clean_db - noise_db - noisesnr) / 10)) * noiseaudio)
        noise = numpy.sum(numpy.concatenate(noises,axis=0),axis=0,keepdims=True)
        return noise + audio

class cluster_loader(object):
    def __init__(self, train_list, train_path, **kwargs):        
        self.data_list, self.data_length, self.data_label = [], [], []
        self.train_path = train_path
        lines = open(train_list).read().splitlines()
        # Get the ground-truth labels, that is used to compute the NMI for post-analyze.
        dictkeys = list(set([x.split()[0] for x in lines]))
        dictkeys.sort()
        dictkeys = { key : ii for ii, key in enumerate(dictkeys) }

        for lidx, line in enumerate(lines):
            data = line.split()
            file_name = data[1]
            file_length = float(data[-1])
            speaker_label = dictkeys[data[0]]
            self.data_list.append(file_name)  # Filename
            self.data_length.append(file_length) # Filelength
            self.data_label.append(speaker_label) # GT Speaker label

        # sort the training set by the length of the audios, audio with similar length are saved togethor.
        inds = numpy.array(self.data_length).argsort()
        self.data_list, self.data_length, self.data_label = numpy.array(self.data_list)[inds], \
                                                            numpy.array(self.data_length)[inds], \
                                                            numpy.array(self.data_label)[inds]
        self.minibatch = []
        start = 0
        while True: # Genearte each minibatch, audio with similar length are saved togethor.
            frame_length = self.data_length[start]
            minibatch_size = max(1, int(1600 // frame_length)) 
            end = min(len(self.data_list), start + minibatch_size)
            self.minibatch.append([self.data_list[start:end], frame_length, self.data_label[start:end]])
            if end == len(self.data_list):
                break
            start = end

    def __getitem__(self, index):
        data_lists, frame_length, data_labels = self.minibatch[index] # Get one minibatch
        filenames, labels, segments = [], [], []
        for num in range(len(data_lists)):
            filename = data_lists[num] # Read filename
            label = data_labels[num] # Read GT label
            audio, sr = soundfile.read(os.path.join(self.train_path, filename))
            if len(audio) < int(frame_length * sr):
                shortage    = int(frame_length * sr) - len(audio) + 1
                audio       = numpy.pad(audio, (0, shortage), 'wrap')
            audio = numpy.array(audio[:int(frame_length * sr)]) # Get clean utterance, better for clustering
            segments.append(audio)
            filenames.append(filename)
            labels.append(label)
        segments = torch.FloatTensor(numpy.array(segments))
        return segments, filenames, labels

    def __len__(self):
        return len(self.minibatch)


Overwriting /kaggle/working/Loss-Gated-Learning/Stage2/dataLoader.py


In [5]:
%%writefile /kaggle/working/Loss-Gated-Learning/Stage2/tools.py

import warnings, torch, os, math, numpy
from sklearn.metrics import accuracy_score
from sklearn import metrics
from operator import itemgetter

def print_write(type, text, score_file): # A helper function to print the text and write the log
    if type == 'T': # Classification training without LGL (Baseline)
        epoch, loss, acc, nselects = text
        print("%d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%\n"%(epoch, loss, acc, nselects))
        score_file.write("[T], %d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%\n"%(epoch, loss, acc, nselects))	
    elif type == 'L': # Classification training with LGL (Propose)
        epoch, loss, acc, nselects, gate = text
        print("%d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%, Gate %.1f \n"%(epoch, loss, acc, nselects, gate))
        score_file.write("[L], %d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%, Gate %.1f \n"%(epoch, loss, acc, nselects, gate))	
    elif type == 'C': # Clustering step
        epoch, NMI = text
        print("%d epoch, NMI %.2f\n"%(epoch, NMI))
        score_file.write("[C], %d epoch, NMI %.2f\n"%(epoch, NMI))
    elif type == 'E': # Evaluation step
        epoch, EER, minDCF = text
        print("EER %2.2f%%, minDCF %2.3f%%\n"%(EER, minDCF))
        score_file.write("[E], %d epoch, EER %2.2f%%, minDCF %2.3f%%\n"%(epoch, EER, minDCF))
    score_file.flush()

def check_clustering(score_path, LGL): # Read the score.txt file, judge the next stage
    lines = open(score_path).read().splitlines()

    if LGL == True: # For LGL, the order is 
        # Iteration 1: (C-T-T...-T-L-L...-L-) 
        # Iteration 2: (C-T-T...-T-L-L...-L-) 
        # ...
        EERs_T, epochs_T, EERs_L, epochs_L = [], [], [], []
        iteration = 0
        train_type = 'T'
        for line in lines:
            if line.split(',')[0] == '[C]': # Clear all results after clustering
                EERs_T, EERs_L, epochs_T, epochs_L = [], [], [], []
                train_type = 'T'
                iteration += 1
            elif line.split(',')[0] == '[E]': # Save the evaluation result in this iteration
                epoch = int(line.split(',')[1].split()[0])
                EER = float(line.split(',')[-2].split()[-1][:-1])
                if train_type == 'T':
                    epochs_T.append(epoch) 
                    EERs_T.append(EER) # Result in [T]
                elif train_type == 'L':
                    epochs_L.append(epoch)
                    EERs_L.append(EER) # Result in [L]
            elif line.split(',')[0] == '[T]': # If the stage is [T], record it
                train_type = 'T'
            elif line.split(',')[0] == '[L]': # If the stage is [L], record it
                train_type = 'L'

        if train_type == 'T': # The stage is [T], so need to judge the next step is keeping [T]? Or do LGL for [L] ?
            if len(EERs_T) < 4: # Too short training epoch, keep training
                return 'T', None, None, iteration
            else:
                if EERs_T[-1] > min(EERs_T) and EERs_T[-2] > min(EERs_T) and EERs_T[-3] > min(EERs_T): # Get the best training result already, go LGL
                    best_epoch = epochs_T[EERs_T.index(min(EERs_T))]
                    next_epoch = epochs_T[-1]
                    return 'L', best_epoch, next_epoch, iteration
                else:
                    return 'T', None, None, iteration # EER can still drop, keep training 

        elif train_type == 'L':
            if len(EERs_L) < 4: # Too short training epoch, keep LGL training
                return 'L', None, None, iteration
            else:
                if EERs_L[-1] > min(EERs_L) and EERs_L[-2] > min(EERs_L) and EERs_L[-3] > min(EERs_L): # Get the best LGL result already, go clustering
                    best_epoch = epochs_L[EERs_L.index(min(EERs_L))]
                    next_epoch = epochs_L[-1]
                    return 'C', best_epoch, next_epoch, iteration # Clustering based on the best epoch is more robust
                else:
                    return 'L', None, None, iteration # EER can still drop, keep training 

    else: # Baseline approach without LGL
        EERs_T, epochs_T = [], []
        iteration = 0
        for line in lines:
            if line.split(',')[0] == '[C]': # Clear all results after clustering
                EERs_T, epochs_T = [], []
                iteration += 1
            elif line.split(',')[0] == '[E]': # Save the evaluation result
                epoch = int(line.split(',')[1].split()[0])
                EER = float(line.split(',')[-2].split()[-1][:-1])
                epochs_T.append(epoch)
                EERs_T.append(EER)

        if len(EERs_T) < 4: # Too short training epoch, keep training
            return 'T', None, None, iteration
        else:
            if EERs_T[-1] > min(EERs_T) and EERs_T[-2] > min(EERs_T) and EERs_T[-3] > min(EERs_T): # Get the best training result, go clustering
                best_epoch = epochs_T[EERs_T.index(min(EERs_T))]
                next_epoch = epochs_T[-1]
                return 'C', best_epoch, next_epoch, iteration
            else:
                return 'T', None, None, iteration # EER can still drop, keep training 

def tuneThresholdfromScore(scores, labels, target_fa, target_fr = None):
    fpr, tpr, thresholds = metrics.roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    tunedThreshold = [];
    if target_fr:
        for tfr in target_fr:
            idx = numpy.nanargmin(numpy.absolute((tfr - fnr)))
            tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]])
    for tfa in target_fa:
        idx = numpy.nanargmin(numpy.absolute((tfa - fpr)))
        tunedThreshold.append([thresholds[idx], fpr[idx], fnr[idx]])
    idxE = numpy.nanargmin(numpy.absolute((fnr - fpr)))
    eer  = max(fpr[idxE],fnr[idxE])*100

    return tunedThreshold, eer, fpr, fnr

# Creates a list of false-negative rates, a list of false-positive rates
# and a list of decision thresholds that give those error-rates.
def ComputeErrorRates(scores, labels):

    # Sort the scores from smallest to largest, and also get the corresponding
    # indexes of the sorted scores.  We will treat the sorted scores as the
    # thresholds at which the the error-rates are evaluated.
    sorted_indexes, thresholds = zip(*sorted(
        [(index, threshold) for index, threshold in enumerate(scores)],
        key=itemgetter(1)))
    sorted_labels = []
    labels = [labels[i] for i in sorted_indexes]
    fnrs = []
    fprs = []

    # At the end of this loop, fnrs[i] is the number of errors made by
    # incorrectly rejecting scores less than thresholds[i]. And, fprs[i]
    # is the total number of times that we have correctly accepted scores
    # greater than thresholds[i].
    for i in range(0, len(labels)):
        if i == 0:
            fnrs.append(labels[i])
            fprs.append(1 - labels[i])
        else:
            fnrs.append(fnrs[i-1] + labels[i])
            fprs.append(fprs[i-1] + 1 - labels[i])
    fnrs_norm = sum(labels)
    fprs_norm = len(labels) - fnrs_norm

    # Now divide by the total number of false negative errors to
    # obtain the false positive rates across all thresholds
    fnrs = [x / float(fnrs_norm) for x in fnrs]

    # Divide by the total number of corret positives to get the
    # true positive rate.  Subtract these quantities from 1 to
    # get the false positive rates.
    fprs = [1 - x / float(fprs_norm) for x in fprs]
    return fnrs, fprs, thresholds

# Computes the minimum of the detection cost function.  The comments refer to
# equations in Section 3 of the NIST 2016 Speaker Recognition Evaluation Plan.
def ComputeMinDcf(fnrs, fprs, thresholds, p_target, c_miss, c_fa):
    min_c_det = float("inf")
    min_c_det_threshold = thresholds[0]
    for i in range(0, len(fnrs)):
        # See Equation (2).  it is a weighted sum of false negative
        # and false positive errors.
        c_det = c_miss * fnrs[i] * p_target + c_fa * fprs[i] * (1 - p_target)
        if c_det < min_c_det:
            min_c_det = c_det
            min_c_det_threshold = thresholds[i]
    # See Equations (3) and (4).  Now we normalize the cost.
    c_def = min(c_miss * p_target, c_fa * (1 - p_target))
    min_dcf = min_c_det / c_def
    return min_dcf, min_c_det_threshold

def accuracy(output, target, topk=(1,)):

    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

Overwriting /kaggle/working/Loss-Gated-Learning/Stage2/tools.py


In [6]:
%%writefile /kaggle/working/Loss-Gated-Learning/Stage2/main_train.py

import os, argparse, pickle, glob
from model import *
from dataLoader import *
from tools import *

parser = argparse.ArgumentParser(description = "Loss Gated Learning")
parser.add_argument('--n_cpu',        type=int, default=8)
parser.add_argument('--max_frames',    type=int, default=180)
parser.add_argument('--batch_size',   type=int, default=256)
parser.add_argument('--init_model',   type=str, default="")
parser.add_argument('--save_path',    type=str, default="")
parser.add_argument('--train_list',   type=str, default="",help='Path for Vox2 list, https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/train_list.txt')
parser.add_argument('--val_list',     type=str, default="", help='Path for Vox_O list, https://www.robots.ox.ac.uk/~vgg/data/voxceleb/meta/veri_test2.txt')
parser.add_argument('--train_path',   type=str, default="", help='Path to the Vox2 set')
parser.add_argument('--val_path',     type=str, default="", help='Path to the Vox_O set')
parser.add_argument('--musan_path',   type=str, default="", help='Path to the musan set')
parser.add_argument('--rir_path',     type=str, default="", help='Path to the rir set')
parser.add_argument('--lr',           type=float, default=0.001)
parser.add_argument('--n_cluster',    type=int, default=4000, help='Number of clusters')
parser.add_argument('--test_interval',type=int, default=1)
parser.add_argument('--max_epoch',    type=int, default=100)
parser.add_argument('--LGL',          dest='LGL', action='store_true', help='Use LGL or baseline only')
args = parser.parse_args()

torch.multiprocessing.set_sharing_strategy('file_system')
warnings.filterwarnings("ignore")
inf_max = 10**3
if args.LGL:
    gates = [1, 3, 3, 5, 6] # Set the gates in each iterations, which is different from our paper because we use stronger augmentation in dataloader
else:
    gates = [inf_max, inf_max, inf_max, inf_max, inf_max] # Set the gate as a very large value = No gate (baseline)

args.model_folder = os.path.join(args.save_path, 'model') # Path for the saved models
args.dic_folder   = os.path.join(args.save_path, 'dic') # Path for the saved pseudo label dic
args.score_path   = os.path.join(args.save_path, 'score.txt') # Path for the score file
os.makedirs(args.model_folder, exist_ok = True)
os.makedirs(args.dic_folder, exist_ok = True)
score_file = open(args.score_path, "a+")

stage, best_epoch, next_epoch, iteration = check_clustering(args.score_path, args.LGL) # Check the state of this epoch
print(stage, best_epoch, next_epoch, iteration)

Trainer = trainer(**vars(args)) # Define the framework
modelfiles = glob.glob('%s/model0*.model'%args.model_folder) # Go for all saved model
modelfiles.sort()

if len(modelfiles) >= 1: # Load the previous model
    Trainer.load_parameters(modelfiles[-1])
    args.epoch = int(os.path.splitext(os.path.basename(modelfiles[-1]))[0][6:]) + 1
else:
    args.epoch = 1 # Start from the first epoch
    for items in vars(args): # Save the parameters in args
        score_file.write('%s %s\n'%(items, vars(args)[items]));
    score_file.flush()

if args.epoch == 1:	# Do clustering in the first epoch
    Trainer.load_parameters(args.init_model) # Load the init_model
    clusterLoader = get_Loader(args, cluster_only = True) # Data Loader
    dic_label, NMI = Trainer.cluster_network(loader = clusterLoader, n_cluster = args.n_cluster) # Do clustering
    pickle.dump(dic_label, open(args.dic_folder + "/label%04d.pkl"%args.epoch, "wb")) # Save the pseudo labels
    print_write(type = 'C', text = [args.epoch, NMI], score_file = score_file)

labelfiles = glob.glob('%s/label0*.pkl'%args.dic_folder) # Read the last pseudo labels
labelfiles.sort()
dic_label = pickle.load(open(labelfiles[-1], "rb"))
print("Dic %s loaded!"%labelfiles[-1])
trainLoader, clusterLoader = get_Loader(args, dic_label) # data loader with the pseduo labels

while args.epoch <= args.max_epoch:
    stage, best_epoch, next_epoch, iteration = check_clustering(args.score_path, args.LGL) # Check the state of this epoch
    print(args.epoch, stage, best_epoch, next_epoch, iteration)
    if stage == 'T': # Classification training
        loss, acc, nselects = Trainer.train_network(epoch = args.epoch, loader = trainLoader, gate = inf_max)
        print_write(type = 'T', text = [args.epoch, loss, acc, nselects], score_file = score_file)

    elif stage == 'L': # LGL training
        if best_epoch != None: # LGL start from the best model from 'T' stage
            Trainer.load_parameters('%s/model0%03d.model'%(args.model_folder, best_epoch)) # Load the best model
        loss, acc, nselects = Trainer.train_network(epoch = args.epoch, loader = trainLoader, gate = gates[iteration - 1])
        print_write(type = 'L', text = [args.epoch, loss,  acc, nselects, gates[iteration - 1]], score_file = score_file)

    elif stage == 'C': # Clustering
        iteration += 1
        if iteration > 5: # Maximun iteration is 5
            quit()
        Trainer.load_parameters('%s/model0%03d.model'%(args.model_folder, best_epoch)) # Load the best model
        clusterLoader = get_Loader(args, cluster_only = True) # Cluster loader
        dic_label, NMI = Trainer.cluster_network(loader = clusterLoader, n_cluster = args.n_cluster) # Clustering
        args.epoch = next_epoch
        print_write(type = 'C', text = [args.epoch, NMI], score_file = score_file)
        pickle.dump(dic_label, open(args.dic_folder + "/label%04d.pkl"%args.epoch, "wb")) # Save the pseudo label dic
        print("Dic %s loaded!"%(args.dic_folder + "/label%04d.pkl"%args.epoch))
        Trainer = trainer(**vars(args)) # Define the framework
        Trainer.load_parameters(args.init_model) # Load the init_model
        trainLoader, clusterLoader = get_Loader(args, dic_label) # Get new dataloader with new label dic

    if args.epoch % args.test_interval == 0 and stage != 'C': # evaluation
        Trainer.save_parameters(args.model_folder + "/model%04d.model"%args.epoch) # Save the model
        EER, minDCF = Trainer.eval_network(**vars(args))
        print_write(type = 'E', text = [args.epoch, EER, minDCF], score_file = score_file)

    args.epoch += 1

Overwriting /kaggle/working/Loss-Gated-Learning/Stage2/main_train.py


In [7]:
with open("/kaggle/working/Loss-Gated-Learning/Stage2/tools.py","r") as f:
    a = f.read()
    
print(a)


import warnings, torch, os, math, numpy
from sklearn.metrics import accuracy_score
from sklearn import metrics
from operator import itemgetter

def print_write(type, text, score_file): # A helper function to print the text and write the log
    if type == 'T': # Classification training without LGL (Baseline)
        epoch, loss, acc, nselects = text
        print("%d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%\n"%(epoch, loss, acc, nselects))
        score_file.write("[T], %d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%\n"%(epoch, loss, acc, nselects))	
    elif type == 'L': # Classification training with LGL (Propose)
        epoch, loss, acc, nselects, gate = text
        print("%d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%, Gate %.1f \n"%(epoch, loss, acc, nselects, gate))
        score_file.write("[L], %d epoch, LOSS %f, ACC %.2f%%, nselects %.2f%%, Gate %.1f \n"%(epoch, loss, acc, nselects, gate))	
    elif type == 'C': # Clustering step
        epoch, NMI = text
        print("%

In [8]:
!python /kaggle/working/Loss-Gated-Learning/Stage2/main_train.py --save_path /kaggle/working/exp/LGL \
--max_epoch 45 --batch_size 350 --lr 0.001 --train_list /kaggle/input/list-file/list_file/train.txt \
--val_list /kaggle/input/list-file/list_file/valid.txt --train_path /kaggle/input/data-cluster/data_cluster/data_set \
--val_path /kaggle/input/data-cluster/data_cluster/data_set --musan_path /kaggle/input/musan-noise \
--rir_path /kaggle/input/rirs-noises/RIRS_NOISES/simulated_rirs \
--init_model /kaggle/input/stage2-14/Loss-Gated-Learning/Stage1/checkpoint/model/model000000046.model \
--test_interval 1 --n_cluster 731 --LGL --max_frames 45

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
T None None 0
Model /kaggle/input/stage2-14/Loss-Gated-Learning/Stage1/checkpoint/model/model000000046.model loaded!
100%|█████████████████████████████████████████| 500/500 [07:01<00:00,  1.19it/s]
1 epoch, NMI 80.62

Dic /kaggle/working/exp/LGL/dic/label0001.pkl loaded!
1 T None None 1

1 epoch, LOSS 8.241371, ACC 7.85%, nselects 100.00%

100%|███████████████████████████████████████| 2138/2138 [00:52<00:00, 41.06it/s]
EER 13.07%, minDCF 0.725%

2 T None None 1

2 epoch, LOSS 6.672605, ACC 15.47%, nselects 100.00%

100%|███████████████████████████████████████| 2138/2138 [00:38<00:00, 54.91it/s]
EER 12.73%, minDCF 0.715%

3 T None None 1

3 epoch, LOSS 6.308904, ACC 18.42%, nselects 100.00%

100%|███████████████████████████████████████| 2138

In [9]:
# !rm -r /kaggle/working/exp